In [1]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble* ./

In [2]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [3]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [25]:
def scoring(predictions, y_test):
  fp = 0
  tp = 0
  fn = 0 
  c = 0
  incorrect = []

  for pred, test in zip(predictions, y_test):
    if pred[0] != test[0]:
      incorrect.append(c)
    if pred[0] == 1:
      if pred[0] == test[0]:
        tp += 1
      else:
        fp += 1
    elif pred[0] != test[0]:
      fn += 1
    c += 1

  print(tp, fp, fn)

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)
  print('Precision: ', precision, '\nRecall: ', recall, '\nF1 Score: ', f1)

In [4]:
from tensorflow import keras

model = keras.models.load_model('ensemble_3')

In [5]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')

In [6]:
import numpy as np
import pickle
import os

X, y = pickle.load(open('X.pkl', 'rb')), df[['Meme']].values

In [9]:
## Suffle Dataframe

seed = np.random.randint(0, 100000)  

np.random.seed(seed)  
np.random.shuffle(X)  
np.random.seed(seed)  
np.random.shuffle(y)

In [29]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5).split(X, y)
scores = []
for k, (train, test) in enumerate(kfold):
  score = model.evaluate(X[test], y[test], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
  scores.append(score[1] * 100)
  scoring((model(X[test]) > 0.5).numpy().astype("int32").tolist(), y[test])
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

binary_accuracy: 78.44%
153 63 6
Precision:  0.7083333333333334 
Recall:  0.9622641509433962 
F1 Score:  0.816
binary_accuracy: 83.44%
153 47 6
Precision:  0.765 
Recall:  0.9622641509433962 
F1 Score:  0.852367688022284
binary_accuracy: 79.69%
154 60 5
Precision:  0.719626168224299 
Recall:  0.9685534591194969 
F1 Score:  0.8257372654155496
binary_accuracy: 83.75%
156 49 3
Precision:  0.7609756097560976 
Recall:  0.9811320754716981 
F1 Score:  0.8571428571428572
binary_accuracy: 79.06%
153 60 7
Precision:  0.7183098591549296 
Recall:  0.95625 
F1 Score:  0.8203753351206435
80.87% (+/- 2.26%)


In [ ]:
X_inc = []

for i, val in enumerate(X):
  for j in incorrect:
    if np.all(X_test[j][:1000] == val[:1000]):
      X_inc.append(i)

In [ ]:
print(X_inc)

[22, 125, 138, 256, 260, 265, 294, 311, 332, 417, 450, 456, 462, 489, 502, 504, 520, 533, 536, 545, 566, 589, 609, 613, 622, 629, 652, 700, 708, 742, 803, 840, 851, 868, 891, 911, 993, 1018, 1058, 1071, 1072, 1104, 1121, 1163, 1172, 1219, 1224, 1240, 1266, 1336, 1339, 1357, 1373, 1403, 1448, 1470, 1472, 1509, 1513, 1519, 1588, 1598]
